Привет! Этот ноутбук написан Егором из AIKC <3. Сегодня мы разберем самый примитивный RAG. Для начала, нам нужны библиотеки, с которыми нам придется работать.

tg: @who_is_sleep

AIKC: https://t.me/aiknowledgeclub

In [1]:
!pip install PyMuPDF
!pip install scapy
!pip install sentence-transformers
!pip install bitsandbytes
!pip install --upgrade transformers
!pip install faiss-cpu
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import requests
import os
import fitz
from tqdm.auto import tqdm
from spacy.lang.en import English
from sentence_transformers import SentenceTransformer, util

Проверим доступ у CUDA. Для этого воспользуемся torch.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Теперь нам надо достать данные. Возьмем, например, статью про KAT(Kolmogorov Arnold Transformer), url: https://arxiv.org/abs/2409.10594

In [3]:
path = "KAT.pdf"

if not os.path.exists(path):
    print(f"[INFO] We don't have this file, downloading...")
    
    url = "https://arxiv.org/pdf/2409.10594"
    
    filename = path
    
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(filename, "wb") as file:
            file.write(response.content)
            print(f"File downloaded as {filename}")
    else:
        print(f"Can't download. Status code: {respones.status_code}")
else:
    print(f"File {path} is good!")

File KAT.pdf is good!


Далее, нам нужно сделать из наших данных DataFrame. Давайте сделаем это.

In [4]:
def formatting_text(text: str) -> str:
    return text.replace('\n', ' ').strip()

def open_and_read_pdf(path_pdf: str) -> list[dict]:
    pdf = fitz.open(path_pdf)
    all_info_about_pdf = []
    for number_of_page, page in tqdm(enumerate(pdf)):
        text = formatting_text(page.get_text())
        all_info_about_pdf.append({
            "page_number": number_of_page + 1, # номер страницы
            "page_char_counts": len(text), # колличество символов
            "page_word_counts": len(text.split(" ")), # колличество слов
            "page_sents_counts": len(text.split(". ")), # колличество предложений
            "page_token_counts(approximately)": len(text) / 4, # колличество токенов, почитатьь про это можно тут: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
            "text": text # текст
        })
    return all_info_about_pdf

pdf = open_and_read_pdf(path_pdf=path)

pdf[:2]

0it [00:00, ?it/s]

[{'page_number': 1,
  'page_char_counts': 3032,
  'page_word_counts': 419,
  'page_sents_counts': 33,
  'page_token_counts(approximately)': 758.0,
  'text': 'Kolmogorov–Arnold Transformer Xingyi Yang Xinchao Wang National University of Singapore xyang@u.nus.edu; xinchao@nus.edu.sg Input Emb. Norm Attention Norm MLP Transformer  Input Emb. Norm Attention Norm KAN ViT + KAN ViT KAT* DeiT ViT + KAN KAT Input Emb. Norm Attention Norm GR-KAN KAT(Ours) Figure 1: (Left) Architecture of standard transformer (e.g. ViT), ViT+KAN which substitutes the MLP with a KAN, and our KAT model. In KAT, the MLP layers in transformers are replaced with GR-KAN layers. (Right) Performance on the ImageNet dataset. KAT∗indicates that the model was initialized using a pre-trained ViT. Generally, KAT outperforms both the ViT and DeiT models. ViT+KAN performs poorly on ImageNet-level training. Abstract Transformers stand as the cornerstone of mordern deep learning. Traditionally, these models rely on multi-layer p

Круто, теперь у нас есть словарь, в котором много информации о тексте. NLP инженеры очень любят большое колличество информации о тексте!

Давайте теперь разобъем текст на чанки. Для начала, нам нужно сделать предложения. Для этого воспользуемся sentenizer'ом от spacy.

In [5]:
nlp = English()

nlp.add_pipe("sentencizer")

for item in tqdm(pdf):
    item['sents'] = list(nlp(item['text']).sents)
    
    item['sents'] = [str(sent) for sent in item['sents']]
    
    item['page_sents_count_spacy'] = len(item['sents'])

  0%|          | 0/19 [00:00<?, ?it/s]

In [6]:
pdf[12]

{'page_number': 13,
 'page_char_counts': 4100,
 'page_word_counts': 556,
 'page_sents_counts': 41,
 'page_token_counts(approximately)': 1025.0,
 'text': 'Visualization of Trained Functions An important aspect to examine is the behavior of the trained rational functions. As shown in Figure 7, we plot the functions for KAT-S with 𝑔= 8 across all 12 layers. The results indicate that within each layer, the rational functions exhibit similar trends, while the functions across different layers tend to differ from one another. 6 Conclusion and Future Work In this work, we introduced the Kolmogorov–Arnold Transformer (KAT), a novel architecture that successfully integrates Kolmogorov-Arnold Networks (KANs) into transformers, addressing key challenges associated with large-scale training scenarios. Our proposed Group-Rational KAN (GR-KAN) variant, with its rational activation functions, group-based parameter sharing, and variance-preserving initialization, demonstrated significant improvements 

Nice! Теперь у нас есть предложения, продолжаем работать с текстом

Сделаем DataFrame, чтобы посмотреть на информацию о нашем dataset'е

In [7]:
train_text_df = pd.DataFrame(pdf)

In [8]:
train_text_df.describe()

,page_number,page_char_counts,page_word_counts,page_sents_counts,page_token_counts(approximately),page_sents_count_spacy
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,10.000000,3368.000000,525.947368,34.052632,842.000000,32.684211
std,5.627314,905.488021,203.089874,16.379098,226.372005,16.626794
min,1.000000,1110.000000,182.000000,2.000000,277.500000,2.000000
25%,5.500000,2952.000000,450.500000,27.500000,738.000000,22.500000
50%,10.000000,3268.000000,494.000000,32.000000,817.000000,32.000000
75%,14.500000,3710.000000,555.000000,38.500000,927.500000,39.000000
max,19.000000,5446.000000,1260.000000,66.000000,1361.500000,61.000000


Пора разбивать текст на чанки

Ниже, пример, как мы будем бить на чанки 25 подряд идущих числе с 0 до 24

In [9]:
num_sents_chunk = 5

def split_list_of_text_into_chunks(input_list: list, slice_size: int):
    return [input_list[i: i + slice_size] for i in range(0, len(input_list), slice_size)]

test = list(range(25))
split_list_of_text_into_chunks(test, num_sents_chunk)

[[0, 1, 2, 3, 4],
 [5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14],
 [15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

А теперь сделаем то же самое для нашего текста

In [10]:
for item in tqdm(pdf):
    item['sents_chunks'] = split_list_of_text_into_chunks(input_list=item['sents'], slice_size=num_sents_chunk)
    item['num_chunks'] = len(item['sents_chunks'])

  0%|          | 0/19 [00:00<?, ?it/s]

Продолжаем делать чанки

In [11]:
import re
from tqdm import tqdm

pdf_chunks = []

for i, item in tqdm(enumerate(pdf)):
    if i % 1 == 0:
        print(f"Processed {i} items")
    
    for sent_chunks in item['sents_chunks']:
        chunk_dict = {}
        chunk_dict['page_number'] = item['page_number']
        
        joined_sent_chunk = " ".join(sent_chunks).replace("  ", " ").strip()
        joined_sent_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sent_chunk)
        
        chunk_dict['sents_chunks'] = joined_sent_chunk
        chunk_dict['chunk_char_count'] = len(joined_sent_chunk)
        chunk_dict['chunk_word_count'] = len(joined_sent_chunk.split())
        chunk_dict['chunk_token_count'] = len(joined_sent_chunk) / 4
        
        pdf_chunks.append(chunk_dict)

19it [00:00, 5487.28it/s]

Processed 0 items
Processed 1 items
Processed 2 items
Processed 3 items
Processed 4 items
Processed 5 items
Processed 6 items
Processed 7 items
Processed 8 items
Processed 9 items
Processed 10 items
Processed 11 items
Processed 12 items
Processed 13 items
Processed 14 items
Processed 15 items
Processed 16 items
Processed 17 items
Processed 18 items


In [12]:
pdf_chunks[0]

{'page_number': 1,
 'sents_chunks': 'Kolmogorov–Arnold Transformer Xingyi Yang Xinchao Wang National University of Singapore xyang@u.nus.edu; xinchao@nus.edu.sg Input Emb. Norm Attention Norm MLP Transformer Input Emb. Norm Attention Norm KAN ViT + KAN ViT KAT* DeiT ViT + KAN KAT Input Emb. Norm Attention Norm GR-KAN KAT(Ours) Figure 1: (Left) Architecture of standard transformer (e.g. ViT), ViT+KAN which substitutes the MLP with a KAN, and our KAT model. In KAT, the MLP layers in transformers are replaced with GR-KAN layers. (',
 'chunk_char_count': 496,
 'chunk_word_count': 76,
 'chunk_token_count': 124.0}

Ну вот мы и сделали чанки! Ура ура, мы молодцы! Но все ли чанки нам нужны? Давайте посмотрим на данные о нашем dataset'е

In [13]:
df = pd.DataFrame(pdf_chunks)
df.describe()

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,134.000000,134.000000,134.000000,134.000000
mean,10.052239,477.216418,75.097015,119.304104
std,5.263461,488.618605,109.526450,122.154651
min,1.000000,2.000000,1.000000,0.500000
25%,5.250000,296.500000,41.000000,74.125000
50%,11.000000,420.000000,65.000000,105.000000
75%,15.000000,570.000000,81.000000,142.500000
max,19.000000,5446.000000,1260.000000,1361.500000


Думаю, чанки, в которых у нас меньше 50 токенов нет смысла рассматривать, там мало контектса

**Задача:** Проверить, прав ли я? Для этого напишите в ячейке ниже код, который покажет, в каких чанках у нас < 50 токенов

Снизу ответ, не подсматривайте! Для начала,подумайте сами

In [14]:
#your code here

In [15]:
min_token_length = 50

In [16]:
for index, row in df[df['chunk_token_count'] <= min_token_length].iterrows():
    print(f"Chunk token count: {row['chunk_token_count']} | Text: {row['sents_chunks']}")

Chunk token count: 30.5 | Text: At their core, transformers are built upon two fundamental components: attention 1 arXiv:2409.10594v1 [cs. LG] 16 Sep 2024
Chunk token count: 30.5 | Text: Í𝑑𝑖𝑛 𝑖=1 𝜙𝑖,𝑑𝑜𝑢𝑡(𝑥𝑖)  , where Φ =  𝜙1,1(·) · · · 𝜙1,𝑑in(·) ... ... ... 𝜙𝑑out,1(·) · · · 𝜙𝑑out,𝑑in(·)  (4) 3
Chunk token count: 31.5 | Text: In practice, we share the parameter for the rational function 𝐹for each group; however, each edge retains a unique scalar 𝑤. 6
Chunk token count: 16.0 | Text: 𝜙!,! 𝜙!,# 𝜙!,$ 𝜙!,% 𝜙#,! 𝜙#,# 𝜙#,$ 𝜙#,% 𝜙$,! 𝜙$,# 𝜙$,$ 𝜙$,% 𝜙%,!
Chunk token count: 30.25 | Text: 𝜙%,# 𝜙%,$ 𝜙%,% 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 𝜙 Input Channels Output Channels Vanilla KAN Pre-Act MLP 𝜙&! 𝜙&! 𝜙&! 𝜙&! 𝜙&!
Chunk token count: 20.5 | Text: The comparison of the number of parameters and computation is listed in Table 2. 7
Chunk token count: 23.25 | Text: IN-1k Top-1 Identity Identity 69.7 Swish Swish 74.4 Identity GeLU 74.5 Identity Swish 74.6 12
Chunk token count: 0.5 | Text: 13
Chu

In [17]:
pdf_chunks_lower_than_50_tokens = df[df['chunk_token_count'] > min_token_length].to_dict(orient='records')
pdf_chunks_lower_than_50_tokens[:2]

[{'page_number': 1,
  'sents_chunks': 'Kolmogorov–Arnold Transformer Xingyi Yang Xinchao Wang National University of Singapore xyang@u.nus.edu; xinchao@nus.edu.sg Input Emb. Norm Attention Norm MLP Transformer Input Emb. Norm Attention Norm KAN ViT + KAN ViT KAT* DeiT ViT + KAN KAT Input Emb. Norm Attention Norm GR-KAN KAT(Ours) Figure 1: (Left) Architecture of standard transformer (e.g. ViT), ViT+KAN which substitutes the MLP with a KAN, and our KAT model. In KAT, the MLP layers in transformers are replaced with GR-KAN layers. (',
  'chunk_char_count': 496,
  'chunk_word_count': 76,
  'chunk_token_count': 124.0},
 {'page_number': 1,
  'sents_chunks': 'Right) Performance on the ImageNet dataset. KAT∗indicates that the model was initialized using a pre-trained ViT. Generally, KAT outperforms both the ViT and DeiT models. ViT+KAN performs poorly on ImageNet-level training. Abstract Transformers stand as the cornerstone of mordern deep learning. Traditionally, these models rely on multi-l

Ну а теперь самое интересное, давайте делать вектора! Для этого используем библиотеку sentence-transformers

In [18]:
emb_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2', device='cpu')

In [19]:
for item in tqdm(pdf_chunks_lower_than_50_tokens):
    item['emb'] = emb_model.encode(item['sents_chunks'], convert_to_tensor=False)

  0%|          | 0/118 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 1/118 [00:00<00:27,  4.25it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  2%|▏         | 2/118 [00:00<00:21,  5.33it/s]

...

 98%|█████████▊| 116/118 [00:21<00:00,  5.60it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 99%|█████████▉| 117/118 [00:21<00:00,  5.11it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 118/118 [00:21<00:00,  5.38it/s]


In [20]:
pdf_chunks_lower_than_50_tokens[0]

{'page_number': 1,
 'sents_chunks': 'Kolmogorov–Arnold Transformer Xingyi Yang Xinchao Wang National University of Singapore xyang@u.nus.edu; xinchao@nus.edu.sg Input Emb. Norm Attention Norm MLP Transformer Input Emb. Norm Attention Norm KAN ViT + KAN ViT KAT* DeiT ViT + KAN KAT Input Emb. Norm Attention Norm GR-KAN KAT(Ours) Figure 1: (Left) Architecture of standard transformer (e.g. ViT), ViT+KAN which substitutes the MLP with a KAN, and our KAT model. In KAT, the MLP layers in transformers are replaced with GR-KAN layers. (',
 'chunk_char_count': 496,
 'chunk_word_count': 76,
 'chunk_token_count': 124.0,
 'emb': array([ 6.57844450e-03, -6.12231344e-02, -5.16507681e-03, -1.08807264e-02,
        -1.97361633e-02, -1.90196175e-03, -1.62605953e-03,  8.01866874e-03,
        -3.61742303e-02, -8.98579787e-03, -1.86397918e-02,  1.54045913e-02,
         9.22136847e-03,  2.02582087e-02,  4.54471484e-02, -3.32390890e-02,
        -1.93106495e-02,  1.74438264e-02, -7.57664517e-02, -5.12021668e-0

Замечательно! Теперь у нас есть векторы чанков!

Давайте сохраним получившийся dataset в csv

In [21]:
text_and_embs = pd.DataFrame(pdf_chunks_lower_than_50_tokens)

text_and_embs.to_csv('/kaggle/working/text_and_embs.csv', index=False)

In [22]:
text_and_embs = pd.read_csv('/kaggle/working/text_and_embs.csv')

Из-за специфики csv(все сохранять как str) обработаем вектора

In [23]:
text_and_embs['emb'] = text_and_embs['emb'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))

In [24]:
text_and_embs['emb'] 

0      [0.0065784445, -0.0612231344, -0.00516507681, ...
1      [-0.00493205618, 0.0052231974, -0.00623739418,...
2      [-0.03309853, 0.0719859153, -0.0240339786, 0.0...
3      [-0.0358217433, 0.0798559263, 2.09089831e-05, ...
4      [-0.0182348043, 0.0463021733, -0.0221040808, 0...
                             ...                        
113    [-0.0460388325, -0.0330894254, -0.0186878126, ...
114    [0.00267366227, -0.0645506606, -0.0324550197, ...
115    [-0.0228865519, -0.0326952115, 0.00180980505, ...
116    [0.0235033557, -0.10433425, 0.0111487489, 0.02...
117    [-0.0124384947, -0.0457091257, -0.0514921285, ...
Name: emb, Length: 118, dtype: object

In [25]:
embeds = torch.tensor(np.array((text_and_embs['emb'].tolist())), dtype=torch.float32).to(device)
embeds[1]

tensor([-4.9321e-03,  5.2232e-03, -6.2374e-03, -6.8223e-03, -4.2219e-03,
        -3.9679e-03,  1.0500e-02, -5.0458e-03, -6.3188e-02,  6.7887e-03,
        -3.9027e-02,  1.1444e-02, -5.8736e-03,  2.3388e-02,  2.4911e-02,
        -4.1258e-02,  2.2229e-02,  4.8294e-02, -1.0745e-01, -4.3620e-02,
        -2.0787e-02, -6.2074e-02, -2.1381e-02,  3.3273e-02, -1.6720e-02,
         6.3777e-03, -9.4501e-03,  2.1166e-02,  4.1475e-03, -1.4575e-02,
         4.0760e-02, -5.7001e-03, -2.3356e-02,  3.6981e-02,  2.2433e-06,
         9.3525e-03,  3.4099e-02, -2.0714e-03, -2.4955e-03,  5.7343e-02,
        -3.1573e-02, -1.3271e-02, -1.0897e-02, -5.8755e-03, -1.5861e-02,
        -5.6963e-02,  7.8531e-02,  5.3214e-02,  1.0906e-02,  6.0941e-02,
        -2.0867e-03, -2.6766e-02,  2.3060e-02, -2.8427e-02, -4.9710e-02,
        -6.2726e-03, -1.2307e-02, -1.6431e-03, -7.3872e-02, -3.2149e-02,
        -1.7307e-02,  3.6560e-02,  5.7437e-02,  2.2651e-02,  2.1581e-02,
         6.4225e-02, -3.1525e-02, -2.1000e-02,  7.2

Круто, теперь у нас тут два подхода как решать дальше задачу
1) испольщовать util из sentence transformers

2) хайповый FAISS

Ну а мы что, мы AIKC, мы покажем оба варианта!

Для начала возьмем util

In [26]:
query = "Overall Architecture of KAT?"

emb_query = emb_model.encode(query, convert_to_tensor=True).to(device)

dot_scores = util.dot_score(a=emb_query, b=embeds)[0]

top_res = torch.topk(dot_scores, k=5)

top_res

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

torch.return_types.topk(
values=tensor([0.5441, 0.5422, 0.4372, 0.3706, 0.3697], device='cuda:0'),
indices=tensor([ 48,  73,  52, 116,  47], device='cuda:0'))

In [27]:
pdf_chunks_lower_than_50_tokens[48]['emb'].shape

(768,)

Давайте запустим модельку

In [28]:
model_id = "unsloth/gemma-2-2b-it-bnb-4bit"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/opt/conda/lib/python3.10/site-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Напишем промпт и подадим модели!

In [29]:
prompt = """
Привет, ты помогаешь пользователю разбираться в запросах по статьям. Отвечай на его вопросы, опираясь на контекст, отвечай на русском. Не пиши код, а словами объясняй как оно работает
{context}
Запрос пользователя:
{query}
Ans:

"""

query = "What is Kolmogorov Arnold Transformer?"

emb_query = emb_model.encode(query, convert_to_tensor=True).to(device)

dot_scores = util.dot_score(a=emb_query, b=embeds.to(device))[0]

top_res = torch.topk(dot_scores, k=1)

most_sim = int(top_res[1][0])

print(top_res)

prompt = prompt.format(
    context=pdf_chunks_lower_than_50_tokens[int(top_res[1][0])]['sents_chunks'],
    query=query
)


dialogue_template = [
        {"role": "user",
         "content": prompt}
    ]

prompt = tokenizer.apply_chat_template(
    conversation=dialogue_template,
    tokenize=False,
    add_generation_prompt=True
)

input_ids = tokenizer(prompt, return_tensors='pt').to(device)

output_ids = model.generate(
    **input_ids,
    temperature=0.85,
    do_sample=True,
    max_new_tokens=1024
)

out_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

torch.return_types.topk(
values=tensor([0.6301], device='cuda:0'),
indices=tensor([0], device='cuda:0'))


Ответ!

In [30]:
print(f"Query: {query}")
print(f"RAG answer:\m{out_text.replace(prompt, '')}")

Query: What is Kolmogorov Arnold Transformer?
RAG answer:\muser
Привет, ты помогаешь пользователю разбираться в запросах по статьям. Отвечай на его вопросы, опираясь на контекст, отвечай на русском. Не пиши код, а словами объясняй как оно работает
Kolmogorov–Arnold Transformer Xingyi Yang Xinchao Wang National University of Singapore xyang@u.nus.edu; xinchao@nus.edu.sg Input Emb. Norm Attention Norm MLP Transformer Input Emb. Norm Attention Norm KAN ViT + KAN ViT KAT* DeiT ViT + KAN KAT Input Emb. Norm Attention Norm GR-KAN KAT(Ours) Figure 1: (Left) Architecture of standard transformer (e.g. ViT), ViT+KAN which substitutes the MLP with a KAN, and our KAT model. In KAT, the MLP layers in transformers are replaced with GR-KAN layers. (
Запрос пользователя:
What is Kolmogorov Arnold Transformer?
Ans:
model
Kolmogorov–Arnold Transformer - это специальный тип модели машинного обучения, который является улучшенным вариантом традиционного архитектуры Transformers, например, ViT (Vision Trans

Теперь сделаем все с FAISS

In [34]:
import faiss

embeds_np = embeds.cpu().numpy().astype('float32')
faiss.normalize_L2(embeds_np)

index = faiss.IndexFlatIP(768)
index.add(embeds_np)

query = "What is Kolmogorov Arnold Transformer?"
emb_query = emb_model.encode(query, convert_to_tensor=True).cpu().numpy().astype('float32')
emb_query = np.expand_dims(emb_query, axis=0)
faiss.normalize_L2(emb_query)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(768,)
(1, 768)


In [35]:
D, I = index.search(emb_query, 1)  # Топ-1 результат

most_sim = int(I[0][0])

prompt = """
Привет, ты помогаешь пользователю разбираться в запросах по статьям. Отвечай на его вопросы, опираясь на контекст, отвечай на русском. Не пиши код, а словами объясняй как оно работает.
{context}
Запрос пользователя:
{query}
Ans:
"""

prompt = prompt.format(
    context=pdf_chunks_lower_than_50_tokens[most_sim]['sents_chunks'],
    query=query
)

dialogue_template = [
    {"role": "user", "content": prompt}
]

prompt = tokenizer.apply_chat_template(
    conversation=dialogue_template,
    tokenize=False,
    add_generation_prompt=True
)

input_ids = tokenizer(prompt, return_tensors='pt').to(device)
output_ids = model.generate(
    **input_ids,
    temperature=0.85,
    do_sample=True,
    max_new_tokens=1024
)

out_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(out_text)

user
Привет, ты помогаешь пользователю разбираться в запросах по статьям. Отвечай на его вопросы, опираясь на контекст, отвечай на русском. Не пиши код, а словами объясняй как оно работает.
Kolmogorov–Arnold Transformer Xingyi Yang Xinchao Wang National University of Singapore xyang@u.nus.edu; xinchao@nus.edu.sg Input Emb. Norm Attention Norm MLP Transformer Input Emb. Norm Attention Norm KAN ViT + KAN ViT KAT* DeiT ViT + KAN KAT Input Emb. Norm Attention Norm GR-KAN KAT(Ours) Figure 1: (Left) Architecture of standard transformer (e.g. ViT), ViT+KAN which substitutes the MLP with a KAN, and our KAT model. In KAT, the MLP layers in transformers are replaced with GR-KAN layers. (
Запрос пользователя:
What is Kolmogorov Arnold Transformer?
Ans:
model
"Kolmogorov-Arnold Transformer" - это архитектура нейросетей, разработанная Xingyi Yang, Xinchao Wang.  

Это модель, которая предлагает альтернативный подход к архитектуре Transformer, известной как "ViT".  

Вместо стандартных MLP (Fully Co

Вот и весь RAG! Если есть вопросы - мой тг я оставил в начале!